# User routes on the site
## Description
**Clickstream** is a sequence of user actions on a website. It allows you to understand how users interact with the site. In this task, you need to find the most frequent custom routes.

## Input data
Input data is а table with clickstream data in file `hdfs:/data/clickstream.csv`.

### Table structure
* `user_id (int)` - Unique user identifier.
* `session_id (int)` - Unique identifier for the user session. The user's session lasts until the identifier changes.
* `event_type (string)` - Event type from the list:
    * **page** - visit to the page
    * **event** - any action on the page
    * <b>&lt;custom&gt;</b> - string with any other type
* `event_type (string)` - Page on the site.
* `timestamp (int)` - Unix-timestamp of action.

### Browser errors
Errors can sometimes occur in the user's browser - after such an error appears, we can no longer trust the data of this session and all the following lines after the error or at the same time with it are considered corrupted and **should not be counted** in statistics.

When an error occurs on the page, a random string containing the word **error** will be written to the `event_type` field.

### Sample of user session
<pre>
+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1620494781|
|    562|       507|       event|      main|1620494788|
|    562|       507|       event|      main|1620494798|
|    562|       507|        page|    family|1620494820|
|    562|       507|       event|    family|1620494828|
|    562|       507|        page|      main|1620494848|
|    562|       507|wNaxLlerrorU|      main|1620494865|
|    562|       507|       event|      main|1620494873|
|    562|       507|        page|      news|1620494875|
|    562|       507|        page|   tariffs|1620494876|
|    562|       507|       event|   tariffs|1620494884|
|    562|       514|        page|      main|1620728918|
|    562|       514|       event|      main|1620729174|
|    562|       514|        page|   archive|1620729674|
|    562|       514|        page|     bonus|1620729797|
|    562|       514|        page|   tariffs|1620731090|
|    562|       514|       event|   tariffs|1620731187|
+-------+----------+------------+----------+----------+
</pre>

#### Correct user routes for a given user:
* **Session 507**: main-family-main
* **Session 514**: main-archive-bonus-tariffs

Route elements are ordered by the time they appear in the clickstream, from earliest to latest.

The route must be accounted for completely before the end of the session or an error in the session.

## Task
You need to use the Spark SQL, Spark RDD and Spark DF interfaces to create a solution file, the lines of which contain **the 30 most frequent user routes** on the site.

Each line of the file should contain the `route` and `count` values **separated by tabs**, where:
* `route` - route on the site, consisting of pages separated by "-".
* `count` - the number of user sessions in which this route was.

The lines must be **ordered in descending order** of the `count` field.

## Criteria
You can get maximum of 3.5 points (final grade) for this assignment, depedning on the number of interface you manage to leverage. The criteria are as follows:

* 0.5 points – Spark SQL solution with 1 query
* 0.5 points – Spark SQL solution with <=2 queries
* 0.5 points – Spark RDD solution
* 0.5 points – Spark DF solution
* 0.5 points – your solution algorithm is relatively optimized, i.e.: no O^2 or O^3 complexities; appropriate object usage; no data leaks etc. This is evaluated by staff.
* 1 point – 1 on 1 screening session. During this session staff member can ask you questions regarding your solution logic, framework usage, questionable parts of your code etc. If your code is clean enough, the staff member can just ask you to solve a theoretical problem connected to Spark.


In [1]:
!pip install pyspark==3.2.4

In [2]:
# Import necessary libraries
import findspark, pyspark
findspark.init()

from pyspark.sql import SparkSession, functions as F
from pyspark.sql.window import Window
from pyspark import SparkContext
from itertools import groupby


In [3]:
# Initialize SparkSession and SparkContext
spark = SparkSession.builder.appName("UserRoutes").getOrCreate()
sc = SparkContext.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2024-10-20 20:43:43,275 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [4]:
spark

In [5]:
sc

<SparkContext master=yarn appName=UserRoutes>

In [6]:
! hdfs dfs -ls -h /data


Found 2 items
-rw-r--r--   1 root supergroup     30.7 M 2023-09-24 20:38 /data/clickstream.csv
drwxr-xr-x   - root supergroup          0 2023-09-24 20:38 /data/transactions


In [7]:
! hdfs dfs -head /data/clickstream.csv


user_id	session_id	event_type	event_page	timestamp
562	507	page	main	1695584127
562	507	event	main	1695584134
562	507	event	main	1695584144
562	507	event	main	1695584147
562	507	wNaxLlerrorU	main	1695584154
562	507	event	main	1695584154
562	507	event	main	1695584154
562	507	event	main	1695584160
562	507	page	rabota	1695584166
562	507	event	rabota	1695584174
562	507	event	rabota	1695584181
562	507	event	rabota	1695584189
562	507	page	main	1695584194
562	507	event	main	1695584204
562	507	event	main	1695584211
562	507	event	main	1695584211
562	507	event	main	1695584219
562	507	page	bonus	1695584221
562	507	page	online	1695584222
562	507	event	online	1695584230
3539	849	page	main	1695584238
3539	849	event	main	1695584252
3539	849	page	online	1695584261
3539	849	page	bonus	1695584269
3539	849	event	bonus	1695584278
3539	849	page	news	1695584285
3539	849	page	main	1695584291
3539	849	event	main	1695584301
3539	849	page	news	1695584306
3539	849	event	news	1695584307
3539	849	page	vklad	169558

In [8]:
# Step 1: Load Data from HDFS
data_path = "hdfs:///data/clickstream.csv"
clickstream_df = spark.read.option("header", "true").option("delimiter", "\t").csv(data_path)

In [9]:
# Register DataFrame as a temporary SQL table
clickstream_df.createOrReplaceTempView("clickstream")

 ## Spark SQL solution

In [10]:
# Step 2: SQL-Based Approach for Cleaning and Route Extraction

# SQL Query to flag errors, remove them, and extract the route per session
sql_query = """
WITH ErrorFlagged AS (
    SELECT
        user_id,
        session_id,
        event_page,
        timestamp,
        MAX(CASE WHEN event_type LIKE '%error%' THEN 1 ELSE 0 END) 
            OVER (PARTITION BY user_id, session_id ORDER BY timestamp) AS error_flag,
        LAG(event_page) OVER (PARTITION BY user_id, session_id ORDER BY timestamp) AS previous_page
    FROM clickstream
),
FilteredRoutes AS (
    SELECT
        user_id,
        session_id,
        event_page,
        timestamp
    FROM ErrorFlagged
    WHERE error_flag = 0 AND (previous_page IS NULL OR previous_page != event_page)
),
AggregatedRoutes AS (
    SELECT
        user_id,
        session_id,
        COLLECT_LIST(event_page) AS route
    FROM FilteredRoutes
    GROUP BY user_id, session_id
)
SELECT
    CONCAT_WS('-', route) AS route_string,
    COUNT(*) AS count
FROM AggregatedRoutes
GROUP BY route_string
ORDER BY count DESC
LIMIT 30
"""

In [11]:
# Execute the SQL query
sql_result = spark.sql(sql_query)
sql_result.show(30)

+--------------------+-----+
|        route_string|count|
+--------------------+-----+
|                main| 8184|
|        main-archive| 1112|
|         main-rabota| 1047|
|       main-internet|  897|
|          main-bonus|  870|
|           main-news|  769|
|        main-tariffs|  677|
|         main-online|  587|
|          main-vklad|  517|
| main-rabota-archive|  170|
| main-archive-rabota|  167|
|  main-bonus-archive|  143|
|   main-rabota-bonus|  139|
|   main-bonus-rabota|  135|
|    main-news-rabota|  135|
|main-archive-inte...|  131|
|main-internet-rabota|  129|
|    main-rabota-news|  129|
|   main-archive-news|  126|
|main-rabota-internet|  124|
|main-internet-arc...|  123|
|  main-archive-bonus|  117|
| main-internet-bonus|  115|
|main-tariffs-inte...|  114|
|   main-news-archive|  113|
|  main-news-internet|  109|
|main-archive-tariffs|  104|
|  main-internet-news|  103|
|main-tariffs-archive|  103|
|    main-rabota-main|   94|
+--------------------+-----+



## Spark DF solution

In [12]:
# Step 3: DataFrame Approach for Cleaning and Route Extraction

# 3.1 - Identify errors using a window function
event_window = Window.partitionBy("user_id", "session_id").orderBy("timestamp")
cleaned_df = (
    clickstream_df.withColumn("error_flag", F.max(F.when(F.col("event_type").rlike(".*[Ee]rror.*"), 1).otherwise(0)).over(event_window))
                  .filter(F.col("error_flag") == 0)
)


In [13]:
# 3.2 - Filter out consecutive duplicate pages within a session
cleaned_df = (
    cleaned_df.withColumn("duplicate_flag", 
                          F.when(F.lag("event_page").over(event_window) == F.col("event_page"), 1).otherwise(0))
              .filter(F.col("duplicate_flag") == 0)
)


In [14]:
# Step 4: Create and Aggregate Routes in DataFrame Approach
routes_window = Window.partitionBy("user_id", "session_id").orderBy("timestamp")
final_routes = (
    cleaned_df.filter(F.col("event_type") == "page")
              .withColumn("route", F.collect_list("event_page").over(routes_window))
              .groupBy("user_id", "session_id")
              .agg(F.max("route").alias("route"))
              .withColumn("route_string", F.concat_ws("-", "route"))
)

In [15]:
# Calculate and Display the 30 Most Frequent Routes
result_df = (
    final_routes.groupBy("route_string")
                .count()
                .orderBy(F.desc("count"))
                .limit(30)
)

In [16]:
result_df.show(30)

+--------------------+-----+
|        route_string|count|
+--------------------+-----+
|                main| 8184|
|        main-archive| 1113|
|         main-rabota| 1047|
|       main-internet|  897|
|          main-bonus|  870|
|           main-news|  769|
|        main-tariffs|  677|
|         main-online|  587|
|          main-vklad|  518|
| main-rabota-archive|  170|
| main-archive-rabota|  167|
|  main-bonus-archive|  143|
|   main-rabota-bonus|  139|
|   main-bonus-rabota|  135|
|    main-news-rabota|  135|
|main-archive-inte...|  132|
|    main-rabota-news|  130|
|main-internet-rabota|  129|
|   main-archive-news|  126|
|main-rabota-internet|  124|
|main-internet-arc...|  123|
|  main-archive-bonus|  117|
| main-internet-bonus|  115|
|main-tariffs-inte...|  114|
|   main-news-archive|  113|
|  main-news-internet|  109|
|main-archive-tariffs|  104|
|  main-internet-news|  103|
|main-tariffs-archive|  103|
|    main-rabota-main|   94|
+--------------------+-----+



## Spark RDD solution

In [17]:
# Step 5: Alternative RDD Approach for Understanding the Logic

# Create RDD from DataFrame
rdd = clickstream_df.rdd

In [18]:
# Identify errors in RDD
error_events = (
    rdd.filter(lambda row: 'error' in row['event_type'])
       .map(lambda row: (f"{row['user_id']}_{row['session_id']}", int(row['timestamp'])))
       .groupByKey()
       .map(lambda entry: (entry[0], min(entry[1])))
       .collectAsMap()
)

In [19]:
# Broadcast error events to all nodes
error_events_broadcast = sc.broadcast(error_events)

In [20]:
# Function to remove consecutive duplicates from a list of pages
def remove_consecutive_duplicates(entry):
    sorted_pages = [page for page, _ in sorted(entry[1], key=lambda x: x[1])]
    pages = [k for k, _ in groupby(sorted_pages)]
    return entry[0], pages

In [21]:
# Process RDD to extract and count routes
routes_rdd = (
    rdd.map(lambda row: (f"{row['user_id']}_{row['session_id']}", (row['event_type'], row['event_page'], int(row['timestamp']))))
       .filter(lambda entry: entry[0] not in error_events_broadcast.value or entry[1][2] < error_events_broadcast.value[entry[0]])
       .filter(lambda entry: entry[1][0] == "page")
       .map(lambda entry: (entry[0], (entry[1][1], entry[1][2])))
       .groupByKey()
       .map(remove_consecutive_duplicates)
       .map(lambda entry: (entry[0], "-".join(entry[1])))
       .map(lambda entry: (entry[1], 1))
       .reduceByKey(lambda a, b: a + b)
       .sortBy(lambda x: x[1], ascending=False)
       .take(30)
)

In [22]:
# Display RDD results
for route, count in routes_rdd:
    print(f"{route}\t{count}")

main	8184
main-archive	1113
main-rabota	1047
main-internet	897
main-bonus	870
main-news	769
main-tariffs	677
main-online	587
main-vklad	518
main-rabota-archive	170
main-archive-rabota	167
main-bonus-archive	143
main-rabota-bonus	139
main-news-rabota	135
main-bonus-rabota	135
main-archive-internet	132
main-rabota-news	130
main-internet-rabota	129
main-archive-news	126
main-rabota-internet	124
main-internet-archive	123
main-archive-bonus	117
main-internet-bonus	115
main-tariffs-internet	114
main-news-archive	113
main-news-internet	109
main-archive-tariffs	104
main-tariffs-archive	103
main-internet-news	103
main-rabota-main	94


In [23]:
# Displaying RDD results in tabular format
rdd_result_df = spark.createDataFrame(routes_rdd, ["route_string", "count"])
rdd_result_df.show(30)

+--------------------+-----+
|        route_string|count|
+--------------------+-----+
|                main| 8184|
|        main-archive| 1113|
|         main-rabota| 1047|
|       main-internet|  897|
|          main-bonus|  870|
|           main-news|  769|
|        main-tariffs|  677|
|         main-online|  587|
|          main-vklad|  518|
| main-rabota-archive|  170|
| main-archive-rabota|  167|
|  main-bonus-archive|  143|
|   main-rabota-bonus|  139|
|    main-news-rabota|  135|
|   main-bonus-rabota|  135|
|main-archive-inte...|  132|
|    main-rabota-news|  130|
|main-internet-rabota|  129|
|   main-archive-news|  126|
|main-rabota-internet|  124|
|main-internet-arc...|  123|
|  main-archive-bonus|  117|
| main-internet-bonus|  115|
|main-tariffs-inte...|  114|
|   main-news-archive|  113|
|  main-news-internet|  109|
|main-archive-tariffs|  104|
|main-tariffs-archive|  103|
|  main-internet-news|  103|
|    main-rabota-main|   94|
+--------------------+-----+



## Checking on the grader

In [24]:
from pyspark.sql import functions as F

# Add an identifier to each result
sql_result_with_id = sql_result.withColumn("method", F.lit("SQL"))
result_df_with_id = result_df.withColumn("method", F.lit("DataFrame"))
rdd_result_df_with_id = rdd_result_df.withColumn("method", F.lit("RDD"))

# Union all three DataFrames together
combined_result = sql_result_with_id.union(result_df_with_id).union(rdd_result_df_with_id)

# Display the combined result table
combined_result.show(90)  # Displaying 30 results for each method (SQL, DataFrame, and RDD)

+--------------------+-----+---------+
|        route_string|count|   method|
+--------------------+-----+---------+
|                main| 8184|      SQL|
|        main-archive| 1112|      SQL|
|         main-rabota| 1047|      SQL|
|       main-internet|  897|      SQL|
|          main-bonus|  870|      SQL|
|           main-news|  769|      SQL|
|        main-tariffs|  677|      SQL|
|         main-online|  587|      SQL|
|          main-vklad|  517|      SQL|
| main-rabota-archive|  170|      SQL|
| main-archive-rabota|  167|      SQL|
|  main-bonus-archive|  143|      SQL|
|   main-rabota-bonus|  139|      SQL|
|   main-bonus-rabota|  135|      SQL|
|    main-news-rabota|  135|      SQL|
|main-archive-inte...|  131|      SQL|
|main-internet-rabota|  129|      SQL|
|    main-rabota-news|  129|      SQL|
|   main-archive-news|  126|      SQL|
|main-rabota-internet|  124|      SQL|
|main-internet-arc...|  123|      SQL|
|  main-archive-bonus|  117|      SQL|
| main-internet-bonus|  1

In [25]:
# Assuming result_df is already defined from the DataFrame-based solution

import json

# Extracting the top 10 frequent routes from the result DataFrame
top_10_routes_df = result_df.limit(10).collect()

# Preparing the dictionary to save as JSON
top_10_routes_dict = {f"{row['route_string']}": row['count'] for index, row in enumerate(top_10_routes_df)}

# Defining the output file path
output_file_path = "result.json"

# Saving the dictionary to a JSON file
with open(output_file_path, 'w') as json_file:
    json.dump(top_10_routes_dict, json_file, indent=4)

# Returning the file path for reference
output_file_path


'result.json'

In [26]:
!curl -F file=@result.json 51.250.123.136:80/MDS-LSML1/Andreevromano/w6/2

0.9999999999999999
Correct main answer!
Correct main-archive answer!
Correct main-rabota answer!
Correct main-internet answer!
Correct main-bonus answer!
Correct main-news answer!
Correct main-tariffs answer!
Correct main-online answer!
Correct main-vklad answer!
Correct main-rabota-archive answer!


In [27]:
# Stop Spark session
spark.stop()